In [1]:
import skgeom as sg
import matplotlib.pyplot as plt
import numpy as np
import logging
import heapq
from euclid3 import *
from itertools import *
from collections import namedtuple

In [2]:
# _window function takes a list, lst and returns a zipped format of iterables prevs, items and next
def _window(lst):
    prevs, items, nexts = tee(lst, 3) 
    prevs = islice(cycle(prevs), len(lst) - 1, None) 
    nexts = islice(cycle(nexts), 1, None) 
    return zip(prevs, items, nexts) 

In [3]:
# Sometimes we might get polygon vertices which lie on the same line or the same point might be repeated again. To avoid that, we normalize the contour. We also convert the vertices to a point instance of Euclid package for geometric manipulation
def _normalize_contour(contour):
    contour = [Point2(float(x), float(y)) for (x, y) in contour]
    return [point for prev, point, next in _window(contour) if not (point == next or (point-prev).normalized() == (next - point).normalized())]

In [4]:
#defining the cross product (AxB) for vectors A(ax,ay) and B(bx,by) 
def _cross(a, b):
    res = a.x * b.y - b.x * a.y
    return res

In [5]:
# To avoid rounding off errors, we have two functions _approximately_equals(for vectors) and _approximately_same(for points). The function will return both the values are equal if the absolute difference between them are less than 1/1000th of the maximum value
def _approximately_equals(a, b):
    return a == b or (abs(a - b) <= max(abs(a), abs(b)) * 0.001)

def _approximately_same(point_a, point_b):
    return _approximately_equals(point_a.x, point_b.x) and _approximately_equals(point_a.y, point_b.y)

In [6]:
# The following class is for split events, a named tuple _SplitEvent is passed into the class
class _SplitEvent(namedtuple("_SplitEvent", "distance, intersection_point, vertex, opposite_edge")):
    __slots__ = ()

    def __lt__(self, other):
        return self.distance < other.distance

    def __str__(self):
        return "{} Split event @ {} from {} to {}".format(self.distance, self.intersection_point, self.vertex, self.opposite_edge)

# The following class is for Edge events, a named tuple _SplitEvent is passed into the class

class _EdgeEvent(namedtuple("_EdgeEvent", "distance intersection_point vertex_a vertex_b")):
    __slots__ = ()

    def __lt__(self, other):
        return self.distance < other.distance

    def __str__(self):
        return "{} Edge event @ {} between {} and {}".format(self.distance, self.intersection_point, self.vertex_a, self.vertex_b)

In [7]:
# Two other namedtuples, _OriginalEdge for storing Original edges with left and right bisector. The other one is Subtree to store the skeleton arcs as source (intersection point/node), height (distance from the event edge), and sinks (neighbours/connections of the source node).

_OriginalEdge = namedtuple("_OriginalEdge", "edge bisector_left, bisector_right")

Subtree = namedtuple("Subtree", "source, height, sinks")

In [8]:
class _LAVertex:
    
    def __init__(self, point, edge_left, edge_right, direction_vectors=None):
        self.point = point
        self.edge_left = edge_left
        self.edge_right = edge_right
        self.prev = None
        self.next = None
        self.lav = None
        self._valid = True             

        creator_vectors = (edge_left.v.normalized()*-1, edge_right.v.normalized())
        if direction_vectors is None:
            direction_vectors = creator_vectors        

        self._is_reflex = ((_cross(*direction_vectors)) < 0) # original polyskel code convention
        

        self._bisector = Ray2(self.point, operator.add(*creator_vectors) * (-1 if self.is_reflex else 1))
        
    @property
    def bisector(self):
        return self._bisector

    @property
    def is_reflex(self):
        return self._is_reflex

    @property
    def original_edges(self):
        # pylint: disable=maybe-no-member
        return self.lav._slav._original_edges

    def next_event(self):
        events = []
        if self.is_reflex:
            # a reflex vertex may generate a split event
            # split events happen when a vertex hits an opposite edge, splitting the polygon in two.
            for edge in self.original_edges: #self.original_edges calls the function original edges inside this _LAVertex class which returns _slav._original_edges
                if edge.edge == self.edge_left or edge.edge == self.edge_right:
                    continue                

                # a potential b is at the intersection of between our own bisector and the bisector of the
                # angle between the tested edge and any one of our own edges.
                
                leftdot = abs(self.edge_left.v.normalized().dot(edge.edge.v.normalized()))
                rightdot = abs(self.edge_right.v.normalized().dot(edge.edge.v.normalized()))
                selfedge = self.edge_left if leftdot < rightdot else self.edge_right 
                otheredge = self.edge_left if leftdot > rightdot else self.edge_right 

                i = Line2(selfedge).intersect(Line2(edge.edge))  
                if i is not None and not _approximately_equals(i, self.point): 
                    # locate candidate b
                    linvec = (self.point - i).normalized() 
                    edvec = edge.edge.v.normalized() 
                    if linvec.dot(edvec) < 0: 
                        edvec = -edvec

                    bisecvec = edvec + linvec 
                    if abs(bisecvec) == 0: 
                        continue
                    bisector = Line2(i, bisecvec) 
                    b = bisector.intersect(self.bisector)   

                    if b is None: 
                        continue

                    # If there is an intersection, we need to check the eligibility of b
                    # a valid intersection b should lie within the area limited by the edge and the bisectors of its two vertices 
                    xleft   = _cross(edge.bisector_left.v.normalized(), (b - edge.bisector_left.p).normalized()) > 0 #sign is reversed because of polyskel convention
                    xright  = _cross(edge.bisector_right.v.normalized(), (b - edge.bisector_right.p).normalized()) < 0 #sign is reversed - polyskel convention
                    xedge   = _cross(edge.edge.v.normalized(), (b - edge.edge.p).normalized()) < 0 #sign is reversed - polyskel convention

                    if not (xleft and xright and xedge): #if any of the conditions are not satisfied, then it is invalid, so move on to the next vertex
                        continue

                    events.append(_SplitEvent(Line2(edge.edge).distance(b), b, self, edge.edge)) #append it to the events using _SplitEvent named tuple

        i_prev = self.bisector.intersect(self.prev.bisector) 
        i_next = self.bisector.intersect(self.next.bisector) 

        if i_prev is not None: 
            events.append(_EdgeEvent(Line2(self.edge_left).distance(i_prev), i_prev, self.prev, self)) 
        if i_next is not None: #i_next exists only for convex vertex
            events.append(_EdgeEvent(Line2(self.edge_right).distance(i_next), i_next, self, self.next)) 
        if not events: #If there are no events, return None
            return None

        ev = min(events, key=lambda event: self.point.distance(event.intersection_point)) #Find the minimum of the events by distance. The procedure for convex events is given in the cell after bisectors handwritten note

        return ev

#method to invalidate the vertex, so it is not considered the next time
    def invalidate(self):
        if self.lav is not None:
            #pylint: disable=too-many-function-args
            self.lav.invalidate(self)
        else:
            self._valid = False

    @property
    def is_valid(self):
        return self._valid

    def __str__(self):
        return "Vertex ({:.2f};{:.2f})".format(self.point.x, self.point.y)

    def __repr__(self):
        return "Vertex ({}) ({:.2f};{:.2f}), bisector {}, edges {} {}".format("reflex" if self.is_reflex else "convex",
                                                                              self.point.x, self.point.y, self.bisector,
                                                                              self.edge_left, self.edge_right)

In [9]:
class _SLAV:
    def __init__(self, polygon, holes):
        contours = [_normalize_contour(polygon)] 
        contours.extend([_normalize_contour(hole) for hole in holes]) 

        self._lavs = [_LAV.from_polygon(contour, self) for contour in contours]        
        self._original_edges = [
            _OriginalEdge(LineSegment2(vertex.prev.point, vertex.point), vertex.prev.bisector, vertex.bisector)
            for vertex in chain.from_iterable(self._lavs)
        ]
# An iterator method which iterates through the list of active vertices (LAV)
    def __iter__(self):
        for lav in self._lavs:
            yield lav
# A method for finding the length of _lavs
    def __len__(self):
        return len(self._lavs)
# Check whether the _lavs is empty or not
    def empty(self):
        return len(self._lavs) == 0
# Handling of edge events
    def handle_edge_event(self, event):
        sinks = [] 
        events = [] 

        lav = event.vertex_a.lav 
        if event.vertex_a.prev == event.vertex_b.next:            
            self._lavs.remove(lav) 
            for vertex in list(lav): 
                sinks.append(vertex.point) 
                vertex.invalidate() 
        else:            
            new_vertex = lav.unify(event.vertex_a, event.vertex_b, event.intersection_point) 
            if lav.head in (event.vertex_a, event.vertex_b):
                lav.head = new_vertex 
            sinks.extend((event.vertex_a.point, event.vertex_b.point))
            next_event = new_vertex.next_event()
            if next_event is not None: 
                events.append(next_event) 

        return (Subtree(event.intersection_point, event.distance, sinks), events) 

    def handle_split_event(self, event):
        lav = event.vertex.lav   
        sinks = [event.vertex.point] #for split event, the only sink will be the reflex vertex
        vertices = []
        x = None  # right vertex
        y = None  # left vertex
        norm = event.opposite_edge.v.normalized() 
        for v in chain.from_iterable(self._lavs):            
            if norm == v.edge_left.v.normalized() and event.opposite_edge.p == v.edge_left.p:
                x = v 
                y = x.prev 
            elif norm == v.edge_right.v.normalized() and event.opposite_edge.p == v.edge_right.p:
                y = v 
                x = y.next 

            if x: 
                xleft   = _cross(y.bisector.v.normalized(), (event.intersection_point - y.point).normalized()) >= 0
                xright  = _cross(x.bisector.v.normalized(), (event.intersection_point - x.point).normalized()) <= 0                

                if xleft and xright: 
                    break
                else:  
                    x = None
                    y = None

        if x is None:            
            return (None, []) 

        v1 = _LAVertex(event.intersection_point, event.vertex.edge_left, event.opposite_edge) 
        v2 = _LAVertex(event.intersection_point, event.opposite_edge, event.vertex.edge_right)

        v1.prev = event.vertex.prev 
        v1.next = x 
        event.vertex.prev.next = v1 
        x.prev = v1 

        v2.prev = y 
        v2.next = event.vertex.next 
        event.vertex.next.prev = v2 
        y.next = v2 

        new_lavs = None 
        self._lavs.remove(lav) 
        if lav != x.lav:            
            self._lavs.remove(x.lav) 
            new_lavs = [_LAV.from_chain(v1, self)] 
        else:
            new_lavs = [_LAV.from_chain(v1, self), _LAV.from_chain(v2, self)] 

        for l in new_lavs:            
            if len(l) > 2:
                self._lavs.append(l) 
                vertices.append(l.head) 
            else:                
                sinks.append(l.head.next.point)  
                for v in list(l): 
                    v.invalidate()

        events = [] 
        for vertex in vertices: 
            next_event = vertex.next_event() 
            if next_event is not None:
                events.append(next_event) 

        event.vertex.invalidate() 
        return (Subtree(event.intersection_point, event.distance, sinks), events) 

In [10]:
class _LAV:
    def __init__(self, slav):
        self.head = None
        self._slav = slav
        self._len = 0     

    @classmethod
    def from_polygon(cls, polygon, slav):
        lav = cls(slav) 
        for prev, point, next in _window(polygon): 
            lav._len += 1 
            vertex = _LAVertex(point, LineSegment2(prev, point), LineSegment2(point, next)) 
            vertex.lav = lav 
            if lav.head is None: 
                lav.head = vertex 
                vertex.prev = vertex.next = vertex 
            else:
                vertex.next = lav.head 
                vertex.prev = lav.head.prev 
                vertex.prev.next = vertex 
                lav.head.prev = vertex                
        return lav 

    @classmethod
    def from_chain(cls, head, slav): 
        lav = cls(slav)
        lav.head = head
        for vertex in lav:
            lav._len += 1
            vertex.lav = lav
        return lav

    def invalidate(self, vertex):
        assert vertex.lav is self, "Tried to invalidate a vertex that's not mine"        
        vertex._valid = False
        if self.head == vertex:
            self.head = self.head.next
        vertex.lav = None

    def unify(self, vertex_a, vertex_b, point):
        replacement = _LAVertex(point, vertex_a.edge_left, vertex_b.edge_right,
                                (vertex_b.bisector.v.normalized(), vertex_a.bisector.v.normalized()))
        replacement.lav = self

        if self.head in [vertex_a, vertex_b]:
            self.head = replacement

        
        vertex_a.prev.next = replacement 
        vertex_b.next.prev = replacement 
        replacement.prev = vertex_a.prev 
        replacement.next = vertex_b.next 

        vertex_a.invalidate()
        vertex_b.invalidate()

        self._len -= 1 
        return replacement 

    def __str__(self):
        return "LAV {}".format(id(self))

    def __repr__(self):
        return "{} = {}".format(str(self), [vertex for vertex in self])

    def __len__(self):
        return self._len

    def __iter__(self):
        cur = self.head
        while True:
            yield cur
            cur = cur.next
            if cur == self.head:
                return

    def _show(self):
        cur = self.head
        while True:
            print(cur.__repr__())
            cur = cur.next
            if cur == self.head:
                break

In [11]:
class _EventQueue:
    def __init__(self):
        self.__data = []

    def put(self, item):
        if item is not None:
            heapq.heappush(self.__data, item) #use heapq, python implementation of heap queue to push the element to the event

    def put_all(self, iterable):
        for item in iterable:
            heapq.heappush(self.__data, item) #heapq push for iterables

    def get(self):
        return heapq.heappop(self.__data) #heapq pop to get the item from queue

    def empty(self):
        return len(self.__data) == 0 #boolean to check whether the heapq is empty or not

    def peek(self):
        return self.__data[0] #get the first item of the eventqueue

    def show(self):
        for item in self.__data: #show the items of the queue
            print(item)

In [12]:
def _merge_sources(skeleton):
    """
    In highly symmetrical shapes with reflex vertices multiple sources may share the same 
    location. This function merges those sources.
    """
    sources = {}
    to_remove = []
    for i, p in enumerate(skeleton):
        source = tuple(i for i in p.source)
        if source in sources:
            source_index = sources[source]
            # source exists, merge sinks
            for sink in p.sinks:
                if sink not in skeleton[source_index].sinks:
                    skeleton[source_index].sinks.append(sink)
            to_remove.append(i)
        else:
            sources[source] = i
    for i in reversed(to_remove):
        skeleton.pop(i)

In [15]:
def skeletonize(polygon, holes=None):
    """
    Compute the straight skeleton of a polygon.

    The polygon should be given as a list of vertices in counter-clockwise order.
    Holes is a list of the contours of the holes, the vertices of which should be in clockwise order.

    Returns the straight skeleton as a list of "subtrees", which are in the form of (source, height, sinks),
    where source is the highest points, height is its height, and sinks are the point connected to the source.
    """
    slav = _SLAV(polygon, holes) 
    output = []
    prioque = _EventQueue() 

    for lav in slav: 
        for vertex in lav:            
            prioque.put(vertex.next_event())
            

    while not (prioque.empty() or slav.empty()):        
        i = prioque.get() 
        if isinstance(i, _EdgeEvent): 
            if not i.vertex_a.is_valid or not i.vertex_b.is_valid:                
                continue

            (arc, events) = slav.handle_edge_event(i) 
        elif isinstance(i, _SplitEvent):
            if not i.vertex.is_valid:                
                continue
            (arc, events) = slav.handle_split_event(i) 

        prioque.put_all(events) 

        if arc is not None:
            output.append(arc)
            
    _merge_sources(output)
    return output

poly_vert = [(40, 50), (40, 520), (625,425), (500,325), (635,250), (635,10), (250,40), (200,200), (100,50)]
#poly_vert = [(100, 50), (200, 200), (250,40), (635,10), (635,250), (500,235), (625,425), (40,520), (40,50)]
skelton_out = skeletonize(poly_vert,[]) 
p_x, p_y = zip(*poly_vert)
plt.plot(p_x,p_y)
plt.plot((p_x[-1],p_x[0]), (p_y[-1],p_y[0]))

for subt in skelton_out:    
    for n in subt.sinks:        
        plt.plot((subt.source.x, n.x), (subt.source.y,n.y))

plt.show()
